# Language Models & Morphological Transduction
In this project studied some traditional appraoches to a few natural language tasks. First, built some n-gram language models on a corpus of Wikipedia articles, and then designed a finite-state transducer for verb conjugation in Spanish.

# Part 1: Language Models

## Preprocessing the Data

* <b>Sentence splitting:</b>&nbsp;&nbsp;&nbsp;&nbsp;In this project, we are interested in modeling individual sentences, rather than longer chunks of text such as paragraphs or documents. The WikiTest dataset provides paragraphs; thus, we provide a simple method to identify individual sentences by splitting paragraphs at punctuation tokens (".",  "!",  "?").

* <b>Sentence markers:</b>&nbsp;&nbsp;&nbsp;&nbsp;For both training and testing corpora, each sentence must be surrounded by a start-of-sentence (`<s>`) and end-of-sentence marker (`/s`). These markers will allow your models to generate sentences that have realistic beginnings and endings.

* <b>Unknown words:</b>&nbsp;&nbsp;&nbsp;&nbsp;In order to deal with unknown words in the test corpora, all words that do not appear in the vocabulary must be replaced with a special token for unknown words (`<UNK>`) before estimating your models. The WikiText dataset has already done this, and you can read about the method in the paper above. When unknown words are encountered in the test corpus, they should be treated as that special token instead.

In [ ]:
# Constants (feel free to use these in your code, but do not change them)
START = "<s>"   # Start-of-sentence token
END = "</s>"    # End-of-sentence-token
UNK = "<UNK>"   # Unknown word token

In [ ]:
import torchtext
import random
import sys

def preprocess(data, vocab=None):
    final_data = []
    lowercase = "abcdefghijklmnopqrstuvwxyz"
    for paragraph in data:
        paragraph = [x if x != '<unk>' else UNK for x in paragraph.split()]
        if vocab is not None:
            paragraph = [x if x in vocab else UNK for x in paragraph]
        if paragraph == [] or paragraph.count('=') >= 2: continue
        sen = []
        prev_punct, prev_quot = False, False
        for word in paragraph:
            if prev_quot:
                if word[0] not in lowercase:
                    final_data.append(sen)
                    sen = []
                    prev_punct, prev_quot = False, False
            if prev_punct:
                if word == '"':
                    prev_punct, prev_quot = False, True
                else:
                    if word[0] not in lowercase:
                        final_data.append(sen)
                        sen = []
                        prev_punct, prev_quot = False, False
            if word in {'.', '?', '!'}: prev_punct = True
            sen += [word]
        if sen[-1] not in {'.', '?', '!', '"'}: continue # Prevent a lot of short sentences
        final_data.append(sen)
    vocab_was_none = vocab is None
    if vocab is None:
        vocab = set()
    for i in range(len(final_data)):
        final_data[i] = [START] + final_data[i] + [END]
        if vocab_was_none:
            for word in final_data[i]:
                vocab.add(word)
    return final_data, vocab

def getDataset():
    dataset = torchtext.datasets.WikiText2(root='.data', split=('train', 'valid'))
    train_dataset, vocab = preprocess(dataset[0])
    test_dataset, _ = preprocess(dataset[1], vocab)
    return train_dataset, test_dataset

train_dataset, test_dataset = getDataset()

In [ ]:
if __name__ == '__main__':
    for x in random.sample(train_dataset, 10):
        print (x)

['<s>', 'Wiśniowiecki', "'s", 'fighting', 'retreat', 'had', 'a', 'major', 'impact', 'on', 'the', 'course', 'of', 'the', 'war', '.', '</s>']
['<s>', '<UNK>', 'from', 'reservoirs', 'significantly', 'reduces', 'the', 'river', "'s", 'runoff', ',', 'causing', 'an', 'annual', 'loss', 'of', 'over', '3', 'million', 'acre', 'feet', '(', '3', '@.@', '7', 'km3', ')', 'from', 'mainstem', 'reservoirs', 'alone', '.', '</s>']
['<s>', 'By', 'the', 'next', 'morning', ',', 'a', 'cold', 'front', 'to', 'the', 'west', 'forced', 'Tanya', 'to', 'accelerate', 'in', 'a', 'more', 'easterly', 'track', '.', '</s>']
['<s>', 'It', 'was', 'written', 'by', 'series', 'creator', 'and', 'executive', 'producer', 'Matthew', 'Weiner', 'and', 'writer', '<UNK>', '<UNK>', ',', 'and', 'directed', 'by', 'Scott', '<UNK>', '.', '</s>']
['<s>', 'A', 'few', 'style', 'guides', 'allow', 'double', 'sentence', 'spacing', 'for', 'draft', 'work', ',', 'and', 'the', 'Gregg', 'Reference', 'Manual', 'makes', 'room', 'for', 'double', 'and', 

## The LanguageModel Class

Implemented 4 types of language models: a <b>unigram</b> model, a <b>smoothed unigram</b> model, a <b>bigram</b> model, and a <b>smoothed bigram</b> model. 

* <b>`__init__(self, trainCorpus)`</b>: Train the language model on `trainCorpus`. This will involve calculating relative frequency estimates according to the type of model you're implementing.

* <b>`generateSentence(self)`</b>: Return a sentence that is generated by the language model. It should be a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>, where each <TT>w<sup>(i)</sup></TT> is a word in your vocabulary (including <TT>&lt;UNK&gt;</TT> but exlcuding <TT>&lt;s&gt;</TT> and <TT>&lt;&sol;s&gt;</TT>). You may assume that <TT>&lt;s&gt;</TT> starts each sentence (with probability $1$). The following words <TT>w<sup>(1)</sup></TT>, ... , <TT>w<sup>(n)</sup></TT>, <TT>&lt;&sol;s&gt;</TT> are generated according to your language model's distribution. Note that the number of words <TT>n</TT> is not fixed; instead, you should stop the sentence as soon as you generate the stop token <TT>&lt;&sol;s&gt;</TT>.

* <b>`getSentenceLogProbability(self, sentence)`</b>:  Return the <em> logarithm of the probability</em> of <TT>sentence</TT>, which is again a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>. You should use the natural logarithm $-$ that is, the base-<em>e</em> logarithm. See the note below about performing your calculations in log space.

* <b>`getCorpusPerplexity(self, testCorpus)`</b>: You need to compute the perplexity (normalized inverse log probability) of `testCorpus` according to your model. For a corpus $W$ with $N$ words and a bigram model, Jurafsky and Martin tells you to compute perplexity as follows: 

$$Perplexity(W) = \Big [ \prod_{i=1}^N \frac{1}{P(w^{(i)}|w^{(i-1)})} \Big ]^{1/N}$$

<b>Implementation Hint:</b> In order to avoid underflow, you will likely need to do all of your calculations in log-space. That is, instead of multiplying probabilities, you should add the logarithms of the probabilities and exponentiate the result:

$$\prod_{i=1}^N P(w^{(i)}|w^{(i-1)}) = \exp\Big (\sum_{i=1}^N \log P(w^{(i)}|w^{(i-1)}) \Big ) $$

Using this property should help you in your implementation of `generateSentence(self)` and `getCorpusPerplexity(self, testCorpus)`.

In [ ]:
import math
import random
from collections import defaultdict

class LanguageModel(object):
    def __init__(self, trainCorpus):
        '''
        Initialize and train the model (i.e. estimate the model's underlying probability
        distribution from the training corpus.)
        '''

        ### DO NOT EDIT ###
        return

    def generateSentence(self):
        '''
        Generate a sentence by drawing words according to the model's probability distribution.
        Note: Think about how to set the length of the sentence in a principled way.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement generateSentence in each subclass.")

    def getSentenceLogProbability(self, sentence):
        '''
        Calculate the log probability of the sentence provided. 
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement getSentenceProbability in each subclass.")
        
    def getCorpusPerplexity(self, testCorpus):
        '''
        Calculate the perplexity of the corpus provided.
        '''

        ### DO NOT EDIT ###
        raise NotImplementedError("Implement getCorpusPerplexity in each subclass.")

    def printSentences(self, n):
        '''
        Prints n sentences generated by your model.
        '''

        ### DO NOT EDIT ###
        for i in range(n):
            sent = self.generateSentence()
            prob = self.getSentenceLogProbability(sent)
            print('Log Probability:', prob , '\tSentence:',sent)

## Unigram Model

Here, Implemented each of the 4 functions described above for an <b>unsmoothed unigram</b> model. The probability distribution of a word is given by $\hat P(w)$.

<font color='green'><b>Hints:</b></font>
* <font color='green'>You should use a <b>dictionary</b> to map tokens to their unigram counts.</font>
* <font color='green'>Since you never want to generate the start-of-sentence token `<s>`, you should <b>not</b> include it in your counts.</font>
* <font color='green'>In general, avoid checking for membership in a list (i.e. avoid `x in lst`). Instead, use sets or dictionaries for this purpose $-$ membership checks are much faster on these data structures.</font>
* <font color='green'>Do <b>not</b> modify the training or test corpora by using `.append(...)` or `.pop(...)` on them. This will cause unexpected behavior in the autograder tests, which do not expect you to be changing the data.

In [ ]:
class UnigramModel(LanguageModel):
  def __init__(self, trainCorpus):
    self.word_count=0
    self.unigram_fre_dic=dict()
    for sentence in trainCorpus:
      for word in sentence:
        if word!="<s>":
          self.word_count+=1
          self.unigram_fre_dic[word]=self.unigram_fre_dic.get(word,0)+1              
                   

  def find_index(self,unigram_word_prob,rand_num):
    index=0
    for i in range(1,len(unigram_word_prob)):
      if unigram_word_prob[i-1]<rand_num and rand_num<=unigram_word_prob[i]:
        index= i-1 
        break   
    return index        

  def generateSentence(self):
      generated_sentence=["<s>"]
      unigram_word_prob=[]
      prev_prob=0
      unigram_word_prob.append(prev_prob)
      for word in self.unigram_fre_dic.keys():
        prev_prob+=self.get_unigram_probability(word)
        unigram_word_prob.append(prev_prob)  
      while(True):
        random_num=random.random()
        word=list(self.unigram_fre_dic.keys())[self.find_index(unigram_word_prob,random_num)]
        generated_sentence.append(word)
        if word=="</s>":
          break       
      return generated_sentence

  def get_unigram_probability(self, word):
    return ((self.unigram_fre_dic[word])/(self.word_count))     

  def getSentenceLogProbability(self, sentence):
    sentence_prob=0
    for word in sentence:
      if word!="<s>":
        sentence_prob+=math.log(self.get_unigram_probability(word))    
    return sentence_prob     

  def getCorpusPerplexity(self, testCorpus):
    prob_logSum_sentence=0
    unigram_count=0
    for sentence in testCorpus:
      unigram_count+=len(sentence)-1
      try:
        prob_logSum_sentence-=self.getSentenceLogProbability(sentence)
      except:
        prob_logSum_sentence-=float('-inf')   
    return math.exp(prob_logSum_sentence/unigram_count)

Testing function that uses very simple training & test corpora. 

In [ ]:
def sanityCheck(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}

    #	Read in the test corpus
    train_corpus = ["By the Late Classic , a network of few <unk> ( few <unk> ) linked various parts of the city , running for several kilometres through its urban core .",
    "Few people realize how difficult it was to create Sonic 's graphics engine , which allowed for the incredible rate of speed the game 's known for ."]
    test_corpus = ["Classic few parts of the game allowed for few <unk> <unk> incredible city .", 
                   "Few <unk> realize the difficult network , which linked the game to Sonic ."]
    train_corpus, _ = preprocess(train_corpus)
    test_corpus, _ = preprocess(test_corpus)
    sentence = preprocess(["Sonic was difficult ."])[0][0]

    # These are the correct answers (don't change them!)
    if model_type == "unigram":
       senprobs = [-19.08542845, -114.5001481799, -108.7963657053, -53.6727664115, -55.4645258807]
       trainPerp, testPerp = 41.3308239726, 38.0122981569
       model = UnigramModel(train_corpus)
    elif model_type == "smoothed-unigram":
       senprobs = [-19.0405293515, -115.3479413049, -108.9114348746, -54.8190029616, -55.8122547346]
       trainPerp, testPerp = 41.9994393615, 39.9531928383
       model = SmoothedUnigramModel(train_corpus)
    elif model_type == "bigram":
       senprobs = [-float('inf'), -10.3450917073, -9.2464794186, -float('inf'), -float('inf')]
       trainPerp, testPerp = 1.3861445461, float('inf')
       model = BigramModel(train_corpus)
    elif model_type == "smoothed-bigram":
       senprobs = [-16.355820202, -76.0026113319, -74.2346475108, -47.2885760372, -51.2730261907]
       trainPerp, testPerp = 12.2307627397, 26.7193157699
       model = SmoothedBigramModelAD(train_corpus)       

    print("--- TEST: generateSentence() ---")
    modelSen = model.generateSentence()
    senTestPassed = isinstance(modelSen, list) and len(modelSen) > 1 and isinstance(modelSen[0], str)
    if senTestPassed:
        print ("Test generateSentence() passed!")
    else:
        print ("Test generateSentence() failed; did not return a list of strings...")

    print("\n--- TEST: getSentenceLogProbability(...) ---")
    sentences = [sentence, *train_corpus, *test_corpus]
    failed = 0
    for i in range(len(sentences)):
        sen, correct_prob = sentences[i], senprobs[i]
        prob = round(model.getSentenceLogProbability(sen), 10)
        print("Correct log prob.:", correct_prob, '\tYour log prob.:', prob, '\t', 'PASSED' if prob == correct_prob else 'FAILED', '\t', sen)
        if prob != correct_prob: failed+=1

    if not failed:
        print ("Test getSentenceProbability(...) passed!")
    else:
        print("Test getSentenceProbability(...) failed on", failed, "sentence" if failed == 1 else 'sentences...')

    print("\n--- TEST: getCorpusPerplexity(...) ---")
    train_perp = round(model.getCorpusPerplexity(train_corpus), 10)
    test_perp = round(model.getCorpusPerplexity(test_corpus), 10)

    print("Correct train perp.:", trainPerp, '\tYour train perp.:', train_perp, '\t', 'PASSED' if trainPerp == train_perp else 'FAILED')
    print("Correct test perp.:", testPerp, '\tYour test perp.:', test_perp, '\t', 'PASSED' if testPerp == test_perp else 'FAILED')
    train_passed, test_passed = train_perp == trainPerp, test_perp == testPerp
    if train_passed and test_passed:
        print("Test getCorpusPerplexity(...) passed!")
    else:
        print("Test getCorpusPerplexity(...) failed on", "the training corpus and the testing corpus..." if not train_passed and not test_passed else "the testing corpus..." if not test_passed else "the training corpus...")

if __name__=='__main__':
    sanityCheck('unigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -19.08542845 	Your log prob.: -19.08542845 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -114.5001481799 	Your log prob.: -114.5001481799 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -108.7963657053 	Your log prob.: -108.7963657053 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -53.6727664115 	Your log prob.: -53.6727664115 	 PASSED 	 ['<s>', 'Classic', 'few', '

Now, training model on the full WikiText corpus, and evaluate it on the held-out test set.

In [ ]:
def runModel(model_type):
    assert model_type in {'unigram', 'bigram', 'smoothed-unigram', 'smoothed-bigram'}
    # Read the corpora
    if model_type == 'unigram':
        model = UnigramModel(train_dataset)
    elif model_type == 'bigram':
        model = BigramModel(train_dataset)
    elif model_type == 'smoothed-unigram':
        model = SmoothedUnigramModel(train_dataset)
    else:
        model = SmoothedBigramModelAD(train_dataset)

    print("--------- 5 sentences from your model ---------")
    model.printSentences(5)

    print ("\n--------- Corpus Perplexities ---------")
    print ("Training Set:", model.getCorpusPerplexity(train_dataset))
    print ("Testing Set:", model.getCorpusPerplexity(test_dataset))

if __name__=='__main__':
    runModel('unigram')

--------- 5 sentences from your model ---------
Log Probability: -153.05715591834075 	Sentence: ['<s>', 'which', 'December', 'freedom', '@-@', '.', 'gun', 'of', 'Hamilton', '000', 'the', 'shown', 'called', 'conflicting', 'Captain', '.', 'Florida', 'by', 'his', 'pattern', 'to', 'by', 'the', '</s>']
Log Probability: -121.76774992684382 	Sentence: ['<s>', 'hour', '(', 'mass', 'other', 'the', 'one', 'agencies', 'video', '?', '<UNK>', 'season', 'Jovian', 'were', 'literary', 'priests', '</s>']
Log Probability: -278.4319904554388 	Sentence: ['<s>', '(', 'Staff', '.', 'Pinochet', '3rd', 'to', 'David', 'Achievement', 'was', '<UNK>', 'single', 'her', 'also', '.', 'study', 'decried', 'Wu', 'song', 'Portugal', 'sold', 'older', 'with', 'west', 'tolerance', 'on', '"', 'heavy', 'Government', '@.@', ',', 'part', ',', 'the', 'intensity', 'Tree', 'praising', '</s>']
Log Probability: -83.39118804549047 	Sentence: ['<s>', 'the', 'Wes', 'perform', "'s", 'and', 'sex', 'architect', 'of', 'downstream', '5', '

## Smoothed Unigram Model

Here, Implemented each of the 4 functions described above for a <b>unigram</b> model with <b>Laplace (add-one) smoothing</b>. The probability distribution of a word is given by $P_L(w)$. This type of smoothing takes away some of the probability mass for observed events and assigns it to unseen events.

In order to smooth the model, needed items are: the number of words in the corpus, $N$, and the number of word types, $S$. The distinction between these is meaningful: $N$ indicates the number of word instances, where $S$ refers to the size of our vocabulary. For example, the sentence <em>the cat saw the dog</em> has four word types (<em>the</em>, <em>cat</em>, <em>saw</em>, <em>dog</em>), but five word tokens (<em>the</em>, <em>cat</em>, <em>saw</em>, <em>the</em>, <em>dog</em>). The token <em>the</em> appears twice in the sentence, but they share the same type <em>the</em>.

If $c(w)$ is the frequency of $w$ in the training data, you can compute $P_L(w)$ as follows:

$$P_L(w)=\frac{c(w)+1}{N+S}$$


In [ ]:
class SmoothedUnigramModel(UnigramModel):
    def __init__(self, trainCorpus):

        UnigramModel.__init__(self,trainCorpus)

    def get_unigram_probability(self, word):
      return ((self.unigram_fre_dic[word]+1)/(self.word_count +len(self.unigram_fre_dic)))

In [ ]:
if __name__=='__main__':
    sanityCheck('smoothed-unigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -19.0405293515 	Your log prob.: -19.0405293515 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -115.3479413049 	Your log prob.: -115.3479413049 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -108.9114348746 	Your log prob.: -108.9114348746 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -54.8190029616 	Your log prob.: -54.8190029616 	 PASSED 	 ['<s>', 'Classic', 'few

In [ ]:
if __name__=='__main__':
    runModel('smoothed-unigram')

--------- 5 sentences from your model ---------
Log Probability: -15.94305245410116 	Sentence: ['<s>', 'bleed', '</s>']
Log Probability: -162.101700803714 	Sentence: ['<s>', 'consists', ',', 'Many', ',', 'series', 'RIAA', 'types', 'Britain', 'profit', 'detailing', 'touchdown', 'a', 'season', 'exact', 'them', 'physically', 'the', 'the', 'Liberal', 'In', '</s>']
Log Probability: -184.53249455480972 	Sentence: ['<s>', 'at', 'therefore', '(', 'against', '<UNK>', 'ESA', 'Cape', '.', 'join', 'Quite', 'from', 'Metallurgical', ')', 'however', ',', 'future', 'to', 'One', '.', 'kn', 'a', 'and', 'actions', 'or', 'had', '</s>']
Log Probability: -453.23231511310496 	Sentence: ['<s>', 'did', 'took', 'Blackwell', 'inspiration', 'wrote', 'two', 'south', 'regularly', 'This', 'belongings', 'about', 'commented', 'rest', 'of', 'talks', '<UNK>', 'including', "'", "'", 'for', 'a', 'Christian', 'job', 'king', 'the', 'of', 'their', 'also', 'of', 'widespread', 'method', 'in', 'projects', 'ceasefire', 'General'

## Bigram Model

Here, Implemented each of the 4 functions described above for an <b>unsmoothed bigram</b> model. The probability distribution of a word is given by $\hat P(w'|w)$. Thus, the probability of $w_i$ is conditioned on $w_{i-1}$.

In [ ]:
class BigramModel(LanguageModel):
  def __init__(self, trainCorpus):
    self.bigram_fre_dict = dict()
    self.unique_bigram = set()
    self.total_word_count=0
    self.unique_word=set()
    self.unigram_fre_dic=dict()
    for sentence in trainCorpus:
        prev_word = "<s>"
        for i in range(1,len(sentence)):
          word=sentence[i]
          self.unique_word.add(word)
          self.total_word_count+=1
          self.unique_word.add(word)
          word_pair=(prev_word, word)
          self.unique_bigram.add(word_pair)
          self.bigram_fre_dict[prev_word]=self.bigram_fre_dict.get(prev_word,{})
          self.bigram_fre_dict[prev_word][word]= self.bigram_fre_dict[prev_word].get(word,0)+1
          self.unigram_fre_dic[word]=self.unigram_fre_dic.get(word,0)+1
          prev_word = word    
    self.num_unique_bigrams = len(self.bigram_fre_dict)

  def find_index(self, bigram_word_prob, rand_num):
      index = 0
      for i in range(1, len(bigram_word_prob)):
          if bigram_word_prob[i - 1] < rand_num and rand_num <= bigram_word_prob[i]:
              index = i - 1
              break
      return index

  def generateSentence(self):
      prev_word="<s>"
      generated_sentence = ["<s>"]
      while (True):
        
        possible_next_words=list(self.bigram_fre_dict[prev_word])
        bigram_word_prob = []
        prev_prob = 0
        bigram_word_prob.append(prev_prob)
        for word in possible_next_words:
            prev_prob += self.bigram_fre_dict[prev_word][word]/sum(self.bigram_fre_dict[prev_word].values())
            bigram_word_prob.append(prev_prob)            
        random_num = random.random()
        next_word=list(self.bigram_fre_dict[prev_word].keys())[self.find_index(bigram_word_prob, random_num)]
        generated_sentence.append(next_word)
        prev_word=next_word    
        if next_word=="</s>":
            break              
      return generated_sentence
  def get_bigram_probability(self, prev_word, next_word):
      next_word_dic=self.bigram_fre_dict.get(prev_word,0)
      if next_word_dic==0:
        numerator=0
      else:  
        numerator = next_word_dic.get(next_word, 0)
        denominator = sum(next_word_dic.values())
      return 0 if numerator == 0 or denominator == 0 else numerator / denominator
  def getSentenceLogProbability(self, sentence):
      sentence_prob = 0
      prev_word = "<s>"
      for i in range(1,len(sentence)):
        prob=self.get_bigram_probability(prev_word, sentence[i])  
        if prob==0:
          sentence_prob +=float('-inf')
          break
        else:
          log_prob=math.log(prob)
          sentence_prob += log_prob
        prev_word = sentence[i]  
      return sentence_prob

  def number_of_bigrams(self, sentences):
      count = 0
      for sentence in sentences:
          count += len(sentence) - 1
      return count

  def getCorpusPerplexity(self, testCorpus):
      prob_logSum_sentence = 0
      bigram_count = self.number_of_bigrams(testCorpus)
      for sentence in testCorpus:
          try:
              prob_logSum_sentence -= self.getSentenceLogProbability(sentence)
          except:
              prob_logSum_sentence -= float('-inf')
      return math.exp(prob_logSum_sentence / bigram_count)

In [ ]:
if __name__=='__main__':
    sanityCheck('bigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -10.3450917073 	Your log prob.: -10.3450917073 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -9.2464794186 	Your log prob.: -9.2464794186 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -inf 	Your log prob.: -inf 	 PASSED 	 ['<s>', 'Classic', 'few', 'parts', 'of', 'the', 'game', 'allowed', 'f

In [ ]:
if __name__=='__main__':
    runModel('bigram')

--------- 5 sentences from your model ---------
Log Probability: -94.55618869906539 	Sentence: ['<s>', 'It', 'has', 'said', 'she', 'never', 'experienced', 'when', 'Danny', 'Smith', 'published', 'a', 'mount', ',', 'which', 'called', 'The', 'president', 'of', 'Korea', '.', '</s>']
Log Probability: -11.955278808204525 	Sentence: ['<s>', 'Bang', '!', '</s>']
Log Probability: -75.35601663729683 	Sentence: ['<s>', 'It', 'was', 'to', 'remain', 'on', 'October', '3', '@.@', '80', 'tackles', 'and', 'the', 'United', 'States', 'Army', '(', '19', '@,@', '000', 'supporters', '.', '</s>']
Log Probability: -91.66879772779825 	Sentence: ['<s>', 'Nevertheless', ',', 'directed', 'by', 'Chris', '<UNK>', 'her', 'return', 'to', 'this', 'episode', ',', 'the', 'city', 'of', 'Frankie', 'Gavin', 'and', '"', 'county', '.', '</s>']
Log Probability: -13.66556561474093 	Sentence: ['<s>', 'Most', 'Valuable', 'Player', 'Award', '.', '</s>']

--------- Corpus Perplexities ---------
Training Set: 76.92394608735728
Test

## Smoothed Bigram Model

Here, Implemented each of the 4 functions described above for a <b>bigram</b> model with <b>absolute discounting</b>. The probability distribution of a word is given by $P_{AD}(w’|w)$.

In order to smooth the model, computeed a discounting factor $D$. If $n_k$ is the number of bigrams $w_1w_2$ that appear exactly $k$ times, you can compute $D$ as: 

$$D=\frac{n_1}{n_1+2n_2}$$ 

For each word $w$, computed the number of bigram types $ww’$ as follows: 

$$S(w)=|\{w’\mid c(ww’)>0\}|$$ 

where $c(ww’)$ is the frequency of $ww’$ in the training data. In other words, $S(w)$ is the number of unique words that follow $w$ at least once in the training data.

Finally, computing $P_{AD}(w’|w)$ as follows: 

$$P_{AD}(w’|w)=\frac{\max \big (c(ww’)-D,0\big )}{c(w)}+\bigg (\frac{D}{c(w)}\cdot S(w) \cdot P_L(w’)\bigg )$$ 

where $c(w)$ is the frequency of $w$ in the training data and $P_L(w’)$ is the Laplace-smoothed unigram probability of $w’$.

In [ ]:
         
class SmoothedBigramModelAD(BigramModel):
  def __init__(self, trainCorpus):
    BigramModel.__init__(self,trainCorpus)
    self.D=0
    n1=0
    n2=0
    for prev_word in self.bigram_fre_dict.keys():
      for next_word in self.bigram_fre_dict[prev_word].keys():
        if self.bigram_fre_dict[prev_word][next_word]==1:
          n1 +=1
        if self.bigram_fre_dict[prev_word][next_word]==2:
          n2 +=1  
    self.D=n1/(n1+2*n2)
      #S(w) - number of unique words that follow w
      #c(ww') - frequency of bigram ww' in train corpus
      #c(w) - frequency of unigram w in train corpus

    self.S_W={}
    self.C_W={}
    for word in self.bigram_fre_dict:
      self.S_W[word]=len(self.bigram_fre_dict[word]) 
      self.C_W[word]=sum(self.bigram_fre_dict[word].values())
    
  def get_Laplace_smoothed_unigram_probability(self, word):
    num_occurance=self.unigram_fre_dic.get(word,0)
    return ((num_occurance+1)/(self.total_word_count +len(self.unique_word)))
 
  def getSentenceLogProbability(self, sentence):
    sentence_prob = 0
    prev_word = "<s>"
    for i in range(1,len(sentence)):
      prob=self.get_bigram_smoothed_probability(prev_word, sentence[i])  
      if prob==0:
        sentence_prob +=float('-inf')
        break
      else:
        log_prob=math.log(prob)
        sentence_prob += log_prob
      prev_word = sentence[i]  
    return sentence_prob

  def get_bigram_smoothed_probability(self,pre_word,next_word):
    freq_bigram=0
    fre_word=self.C_W.get(pre_word,0)
    if fre_word==0:
      return 0
    else:
      freq_bigram=self.bigram_fre_dict[pre_word].get(next_word,0)
    laplase_prob=self.get_Laplace_smoothed_unigram_probability(next_word)
    prob_word=max((freq_bigram-self.D),0)/self.C_W[pre_word] +((self.D/self.C_W[pre_word])*self.S_W[pre_word]*laplase_prob) 
    return prob_word
  

In [ ]:
if __name__=='__main__':
    sanityCheck('smoothed-bigram')

--- TEST: generateSentence() ---
Test generateSentence() passed!

--- TEST: getSentenceLogProbability(...) ---
Correct log prob.: -16.355820202 	Your log prob.: -16.355820202 	 PASSED 	 ['<s>', 'Sonic', 'was', 'difficult', '.', '</s>']
Correct log prob.: -76.0026113319 	Your log prob.: -76.0026113319 	 PASSED 	 ['<s>', 'By', 'the', 'Late', 'Classic', ',', 'a', 'network', 'of', 'few', '<UNK>', '(', 'few', '<UNK>', ')', 'linked', 'various', 'parts', 'of', 'the', 'city', ',', 'running', 'for', 'several', 'kilometres', 'through', 'its', 'urban', 'core', '.', '</s>']
Correct log prob.: -74.2346475108 	Your log prob.: -74.2346475108 	 PASSED 	 ['<s>', 'Few', 'people', 'realize', 'how', 'difficult', 'it', 'was', 'to', 'create', 'Sonic', "'s", 'graphics', 'engine', ',', 'which', 'allowed', 'for', 'the', 'incredible', 'rate', 'of', 'speed', 'the', 'game', "'s", 'known', 'for', '.', '</s>']
Correct log prob.: -47.2885760372 	Your log prob.: -47.2885760372 	 PASSED 	 ['<s>', 'Classic', 'few', 'pa

In [ ]:
if __name__=='__main__':
    runModel('smoothed-bigram')

--------- 5 sentences from your model ---------
Log Probability: -86.90866606296277 	Sentence: ['<s>', 'In', '1613', ',', 'modeled', 'after', 'the', 'raising', 'its', 'greatest', 'basketball', '@-@', 'a', 'solar', 'parallax', 'and', 'Hindenburg', '.', '</s>']
Log Probability: -140.96797193424015 	Sentence: ['<s>', 'As', 'a', 'bill', 'when', '<UNK>', "'s", 'first', '@-@', 'line', 'in', 'use', 'of', 'The', 'Crime', 'Thriller', '(', '21', ',', 'and', 'Devonian', 'period', 'of', 'the', '<UNK>', 'introduced', 'an', 'unfinished', 'at', 'the', 'bottom', '.', '</s>']
Log Probability: -87.50110208673092 	Sentence: ['<s>', 'The', 'Légion', 'd', '<UNK>', 'and', 'power', 'to', 'game', 'after', 'Mfume', 'into', 'double', 'the', 'Order', 'of', 'the', 'Ganges', 'valley', '.', '</s>']
Log Probability: -182.1982262272302 	Sentence: ['<s>', 'At', 'the', 'General', 'to', 'the', 'largest', 'and', 'not', 'checked', 'into', 'the', 'assassination', 'in', 'association', ',', 'elements', 'like', 'recognizing',

# Part 2: Finite State Transducers

Here, Implemented a <b>finite state transducer</b> (FST), which transduces the infinitive form of Spanish verbs to the preterite (past tense) form in the 3rd person singular.

## Helper functions
Here is a description of the most useful methods of this module:


* <b>`FST(self, initialStateName)`</b>: Instantiate an FST with an initial (non-accepting) state named `initialStateName`

* <b>`addState(self, name, isFinal=False)`</b>: Add a state `name` to the FST; by default, `isFinal=False` and so the state is not an accepting state.

* <b>`addTransition(self, inStateName, inString, outString, outStateName)`</b>: Add a transition between state `inStateName` and state `outStateName`, where both of these states already exist in the FST. The FST can traverse this transition after reading `inString`, and outputs `outString` when it does so.

* <b>`addSetTransition(self, inStateName, inStringSet, outStateName)`</b>: Add a transition between state `inStateName` and state `outStateName` for each character in `inStringSet`. For each transition, the FST outputs the same character it reads.

The cell also contains a method to read the data and a scoring function that you will call in the "Test your FST" section.


## Build the FST 
 
The FST will finish its analysis for a string after having read its last character. If it accepts the string, it will output the transduced form; if it does not accept it, the FST will output FAIL. The FST may be non-deterministic (that is, a state may have multiple transitions for the same input character), but each accepted string should only have one analysis. 

In [ ]:
import os
import pandas as pd
from tabulate import tabulate
pdtabulate=lambda df:tabulate(df,headers='keys',tablefmt='psql', showindex=False)

def readVerbFile(file):
    url='https://drive.google.com/u/0/uc?id=18x48rbiNvWoB54wccc635IVkOHzIfS6K&export=download'
    df = pd.read_csv(url, header=None)

    return df.values.tolist()

verbs = readVerbFile('verbsList.csv')

def testFST(print_examples = 'all'):
    assert print_examples in {'all', 'incorrect', 'none'}, "print_examples must be 'all', 'incorrect', or 'none'"
    rule_classes = [('0a*', "0a* Regular -Car verb (stem end in consonant)", 'hablar      ==>  habló'), ('0b*', "0b* Regular -Var verb (stem end in vowel)", 'pasear      ==>  paseó'), ('0c*', "0c* Regular -er verb", 'comer       ==>  comió'), ('0d*', "0d* Regular -ir verb (excluding -guir, -quir)", 'abrir       ==>  abrió'), ('1a', "1a  Verbs in -ñer", 'tañer       ==>  tañó'), ('1b', "1b  Verbs in -ñir (excluding -eñir)", 'gañir       ==>  gañó'), ('2a', "2a  Verbs in -Ver", 'leer        ==>  leyó'), ('2b', "2b  Verbs in -Vir", 'construir   ==>  construyó'), ('2c*', "2c* Verbs in -guir (excluding -eguir)", 'distinguir  ==>  distinguió'), ('2d*', "2d* Verbs in -quir", 'delinquir   ==>  delinquió'), ('3a', "3a  Verbs in -eCir", 'pedir       ==>  pidió'), ('3b', "3b  Verbs in -eCCir", 'sentir      ==>  sintió'), ('3c', "3c  Verbs in -eCCCir", 'henchir     ==>  hinchió'), ('3d', "3d  Verbs in -eguir", 'seguir      ==>  siguió'), ('3e', "3e  Verbs in -eñir", 'heñir       ==>  hiñó')]

    f = buildFST()
    myParses = f.parseInputList([x[0] for x in verbs])
    scores, totals, examples = {}, {}, []

    for i in range(len(verbs)):
        lemma, form, clas = verbs[i]
        output = myParses[i]
        scores[clas] = scores.get(clas, 0)
        totals[clas] = totals.get(clas, 0) + 1
        if print_examples == 'all' or print_examples == 'incorrect' and form != output:
            examples += [(lemma, form, output, 'CORRECT' if form == output else 'INCORRECT')]
        if form == output: scores[clas] += 1
    
    if print_examples != 'none' and len(examples) > 0:
        examples = pd.DataFrame.from_records(examples, columns = ['Input', 'Correct Output', 'Returned Output', 'Result'])
        print(pdtabulate(examples))

    data= []
    
    # We use a scoring method that accounts for (1) the fact that the default FST gets many verbs correct
    #                                           (2) the class inbalance among the different rules
    # p_scores is for verbs that the default FST gets correct, q_scores for verbs it gets wrong
    # Each contains a list of accuracies for each category group (for example, all of the examples where rule 2 applies are in one group)
    p_scores, q_scores = {'0*': [], '2*': []}, {'1': [], '2': [], '3': []}
    for clas, msg, ex in rule_classes:
        acc = scores[clas]/totals[clas]
        data += [(msg, ex, scores[clas], totals[clas], 100*acc)]
        if '0' in clas: p_scores['0*'].append(acc)
        elif '1' in clas: q_scores['1'].append(acc)
        elif '2' in clas and '*' not in clas: q_scores['2'].append(acc)
        elif '2' in clas and '*' in clas: p_scores['2*'].append(acc)
        elif '3' in clas: 
            if clas in {'3a', '3b', '3c'}: q_scores['3'] += [acc, acc, acc] # Weight these higher than -eguir and -eñir
            else: q_scores['3'].append(acc)
        else: assert False, 'should not get here ' + clas

    p_scores = [sum(v) / len(v) for k, v in p_scores.items()] # Get the average for each category group
    q_scores = [sum(v) / len(v) for k, v in q_scores.items()]

    p_scores = [p_scores[0]] * 3 + [p_scores[1]] # Weight 0* higher than 2*

    # Score by averaging the p and q scores, then applying grading formula
    p, q = sum(p_scores) / len(p_scores), sum(q_scores) / len(q_scores)
    final_score = q/2 * (1+p) # Score will be 0 when p is 100% and q is 0%; score will be 100% when p and q are both 100%

    print('\nScorecard:')
    data = pd.DataFrame.from_records(data, columns = ['Category', 'Example', 'Correct', 'Total', 'Accuracy (%)'])
    print(pdtabulate(data))

    print("Overall Score:", str(final_score*100) + '%')

class Transition:
    # string_in
    # string_out
    def __init__(self, inState, inString, outString, outState):
        self.state_in = inState
        self.string_in = inString
        self.string_out = outString
        self.state_out = outState

    def equals(self, t):
        if self.state_in == t.state_in \
        and self.string_in == t.string_in \
        and self.string_out == t.string_out \
        and self.state_out == t.state_out:
            return True
        else:
            return False

class FSTstate:
    # id: an integer ID of the state
    # isFinal: is this a final state?
    def __init__(self, n, isF, fst):
        self.id = n
        self.isFinal = isF
        self.transitions = dict() # map inStrings to a set of all possible transitions
        self.FST = fst

    def addTransition(self, inString, outString, outState):
        newTransition = Transition(self, inString, outString, outState)
        if inString in self.transitions:
            for t in self.transitions[inString]:
                if t.equals(newTransition):
                    return
            self.transitions[inString].add(newTransition)
        else:
            self.transitions[inString] = set([])
            self.transitions[inString].add(newTransition)
    
    def parseInputFromStartState(self, inString):
        parseTuple = ("", self.id)
        parses = []
        (accept, stringParses) = self.parseInput(inString)
        if accept:
            for p in stringParses:
                completeParse = [parseTuple]
                completeParse.extend(p)
                parses.append(completeParse)
        return (accept, parses)

    def parseInput(self, inString):
        parses = []
        isAccepted = True
        
        DEBUG = False
        if DEBUG:
            print("parseInput: state: ", self.id, " parsing: " , inString)
        
        # Case 1: no suffix
        if inString == "":
            epsilonParses = []
            epsilonAccepted = False
            # try all epsilon transitions
            if "" in self.transitions:
                transSet = self.transitions[""]
                for t in transSet:
                    outString = t.string_out
                    toStateID = t.state_out
                    toState = self.FST.allStates[toStateID]
                    parseTuple = (outString, toStateID)
                    (suffixAccepted, suffixParses) = toState.parseInput(inString)
                    if suffixAccepted:
                        epsilonAccepted = True
                        if suffixParses == []: #accepts.
                            parse_s = [parseTuple]
                            epsilonParses.append(parse_s)
                        else:
                            for s in suffixParses:
                                parse_s = [parseTuple]
                                parse_s.extend(s)
                                epsilonParses.append(parse_s)
            # if epsilon is accepted, add all its parses
            if epsilonAccepted:
                parses.extend(epsilonParses)
            # if this is a final state, add an empty parse
            if self.isFinal or parses != []:
                if DEBUG:
                    print("Accepted in state ", self.id)
                return (True, parses)
            else:
                if DEBUG:
                    print("Rejected in state ", self.id)
                return (False, None)
        # case 2: non-empty suffix: there needs to be one suffix that parses!)
        hasAcceptedSuffix = False;
        for i in range(0,len(inString)+1):
            prefix = inString[0:i]
            suffix = inString[i:len(inString)]
            if DEBUG:
                print("\t prefix: \'", prefix, "\' I=", i)
            if prefix in self.transitions:
                if DEBUG:
                     print("\t prefix: ", prefix,  "suffix: ", suffix, "I=", i)
                transSet = self.transitions[prefix]
                for t in transSet:
                    outString = t.string_out
                    toStateID = t.state_out
                    toState = self.FST.allStates[toStateID]
                    parseTuple = (outString, toStateID)
                    (suffixAccepted, suffixParses) = toState.parseInput(suffix)
                    if suffixAccepted:
                        hasAcceptedSuffix = True
                        if suffixParses == []:
                            parse_s = [parseTuple]
                            parses.append(parse_s)
                            thisPrefixParses = True
                        for s in suffixParses:
                            parse_s = [parseTuple]
                            parse_s.extend(s)
                            parses.append(parse_s)
        if hasAcceptedSuffix:
            return (True, parses)
        else:
            return (False, None)
                            


    def printState(self):
        if self.isFinal:
            FINAL = "FINAL"
        else: FINAL = ""
        print("State", self.id, FINAL)
        for inString in self.transitions:
            transList = self.transitions[inString]
            for t in transList:
                print("\t", inString, ":", t.string_out, " => ", t.state_out)

                    


class FST:
    def __init__(self, initialStateName="q0"):
        self.nStates = 0
        self.initState = FSTstate(initialStateName, False, self) 
        self.allStates = dict()
        self.allStates[initialStateName] = self.initState
       
    def addState(self, name, isFinal=False):
        if name in self.allStates:
            print("ERROR addState: state", name, "exists already")
            sys.exit()
        elif len(self.allStates) >= 30:
            print("ERROR addState: you can't have more than 30 states")
            sys.exit()
        else:  
            newState = FSTstate(name, isFinal, self)
            self.allStates[name] = newState

    def addTransition(self, inStateName, inString, outString, outStateName):
        if (len(inString) > 1):
            print("ERROR: addTransition: input string ", inString, " is longer than one character")
            sys.exit()
        if inStateName not in self.allStates:
            print("ERROR: addTransition: state ", inStateName, " does not exist")
            sys.exit()
        if outStateName not in self.allStates:
            print("ERROR: addTransition: state ", outStateName, " does not exist")
            sys.exit()
        inState = self.allStates[inStateName]
        inState.addTransition(inString, outString, outStateName)

    # epsilon:epsilon
    def addEpsilonTransition(self, inStateName, outStateName):
        if inStateName not in self.allStates:
            print("ERROR: addEpsilonTransition: state ", inStateName, " does not exist")
            sys.exit()
        if outStateName not in self.allStates:
            print("ERROR: addEpsilonTransition: state ", outStateName, " does not exist")
            sys.exit()
        if inStateName == outStateName:
            print("ERROR: we don't allow epsilon loops")
            sys.exit()
        inState = self.allStates[inStateName]
        inState.addTransition("", "", outStateName)

    # map every element in inStringSet to itself
    def addSetTransition(self, inStateName, inStringSet, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, s, outStateName)

    # map string to itself
    def addSelfTransition(self, inStateName, inString, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetTransition: state ", outStateName, " does not exist")
            sys.exit()
         self.addTransition(inStateName, inString, inString, outStateName)

    # map every element in inStringSet to outString
    def addSetToStringTransition(self, inStateName, inStringSet, outString, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetDummyTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetDummyTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, outString, outStateName)
    
            
    # map every element in inStirngSet to outString
    def addSetEpsilonTransition(self, inStateName, inStringSet, outStateName):
         if inStateName not in self.allStates:
            print("ERROR: addSetEpsilonTransition: state ", inStateName, " does not exist")
            sys.exit()
         if outStateName not in self.allStates:
            print("ERROR: addSetEpsionTransition: state ", outStateName, " does not exist")
            sys.exit()
         for s in inStringSet:
            self.addTransition(inStateName, s, "", outStateName)
            
    def parseInput(self, inString):
        SHOW_STATES = False#True
        inString = inString.rstrip('\n')
        (canParse, allParses)  = self.initState.parseInputFromStartState(inString)
        allParsesAsString = ""
        if canParse:
            for parse in allParses:
                for tuple in parse:
                    outString, outState = tuple
                    allParsesAsString += outString
                if SHOW_STATES:
                    allParsesAsString += "\t  States: "
                    i = 0
                    for tuple in parse:
                        i += 1
                        outString, outState = tuple
                        allParsesAsString += outState
                        if i < len(parse):
                            allParsesAsString += " => "
                    allParsesAsString += "; "
          
            return True, allParsesAsString
        else:
            return False, "FAIL"

    def printFST(self):
        print("Printing FST", str(self))
        for stateID in self.allStates:
            state = self.allStates[stateID]
            state.printState()

    def parseInputList(self, verbList):
        #with open(fileName, "r") as f:
        nParses = 0
        totalStrings = 0
        res = []
        for verb in verbList:#f:
            totalStrings += 1
            canParse, parse = self.parseInput(verb)
            res += [parse]
            if canParse:
                nParses += 1
        fraction = nParses/totalStrings
        print(nParses, "/", totalStrings, "=", str(fraction*100)+'%', "of examples parsed") 
        return res

## Testing the FST

In [ ]:
# Here are some predefined character sets.
A2Z = set('abcçdefghijklmnopqrstuvwxyzáéíóúñü')
VOWS = set('aeiouáéíóúü')
CONS = A2Z-VOWS

# TODO: Impelement your solution here.
def buildFST():
    l=['g','q']
    g_q_set=set(l)
    # The states (you need to add more)
    f = FST('q1') # q1 is the initial (non-accepting) state
    f.addState('q_r')
    f.addState('q2')
    f.addState('q3')
    f.addState('q4')
    f.addState('q5')
    f.addState('q_epsilon')
    f.addState('q_EOW', True) # An accepting state
    f.addState('q6')
    f.addState('q7')
    f.addState('q8')
    f.addState('q9')
    f.addState('q10')
    f.addState('q11')
    f.addState('q12')

    # The transitions (you need to add more)
    f.addSetTransition('q1', A2Z-{'e'}, 'q1') # Self-loop on all letters to transition through stem
    #f.addSetTransition('q1', A2Z-VOWS-{'ñ','g','q'}, 'q2')
    f.addSetTransition('q1', A2Z-VOWS-{'ñ'}, 'q2')

    f.addTransition('q2', 'a', 'ó', 'q_r') # Transition on vowel of ending, with a ==> ó and e ==> ió and i ==> ió
    f.addTransition('q2', 'e', 'ió', 'q_r')
    f.addTransition('q2', 'i', 'ió', 'q_r')

    #Stems ending in ñ: 
    f.addTransition('q1', 'ñ', 'ñ', 'q3')

    #If the stem ends in ñ, the ending is -ó rather than -ió
    f.addTransition('q3', 'e', 'ó', 'q_r')
    f.addTransition('q3', 'i', 'ó', 'q_r')
    f.addTransition('q3', 'a', 'ó', 'q_r')

    #f.addTransition('q8', 'ñ', 'ñ', 'q3')
    f.addTransition('q8', 'ñ', 'ñ', 'q11')
    f.addTransition('q11', 'i', 'ó', 'q_r')
    f.addTransition('q8', 'g', 'g', 'q6')
    
    #Transition from q10 to q8
    f.addTransition('q10', 'e', 'i', 'q8')
    #Transition from q10 to q9
    f.addTransition('q10', 'e', 'e', 'q9')

    #Stems ending in a vowel: 
    f.addSetTransition('q1', A2Z-{'g','q'}, 'q4')
    f.addSetTransition('q4', VOWS,'q5')
    #If the stem ends in a vowel, the ending is -yó rather than -ió
    f.addTransition('q5', 'i', 'yó', 'q_r')
    f.addTransition('q5', 'e', 'yó', 'q_r')
    f.addTransition('q5', 'a', 'ó', 'q_r')

    #does not apply to verbs ending in -guir or -quir
    f.addSetTransition('q1',g_q_set,'q6')  
    f.addSelfTransition('q6','u','q7')
    f.addTransition('q7', 'i', 'ió', 'q_r')
    f.addTransition('q7', 'e', 'ió', 'q_r')
    f.addTransition('q7', 'a', 'ó', 'q_r')

    #For -ir verbs only, if the stem ends in an e followed by any number of consonants, then the e changes to an i
    f.addTransition('q1', 'e', 'i', 'q8')
    f.addSetTransition('q8', CONS-{'ñ'}, 'q8')
    f.addTransition('q8', 'i', 'ió', 'q_r')

    f.addTransition('q1', 'e', 'e', 'q9')
    f.addSetTransition('q9', CONS, 'q10')
    #f.addSetTransition('q9', CONS-{'g'}, 'q10')
    f.addSetTransition('q10', CONS, 'q10')

    # changing q9 state
    #f.addSetTransition('q9', VOWS, 'q9')

    f.addTransition('q10', 'e', 'ió', 'q_r')
    f.addTransition('q10', 'a', 'ó', 'q_r')

    f.addSetTransition('q10', VOWS-{'e'}, 'q1')
    # transition q10 to q5 
    # trying to change from q10 to q7
    #f.addSetTransition('q10', VOWS, 'q5')
    f.addSetTransition('q10', VOWS, 'q12')
    f.addTransition('q12', 'a', 'ó', 'q_r')
    f.addTransition('q_r', 'r', '', 'q_epsilon') # Transition on final r in infinitive - replace with empty string
    f.addTransition('q_epsilon', '', '', 'q_EOW') # If see empty string, at end of word
    #applying changes after analysis
    f.addTransition('q12', 'e', 'yó', 'q_r')
    f.addSetTransition('q10', VOWS, 'q4')
    f.addSetTransition('q9', VOWS, 'q1')
    f.addTransition('q6', 'i', 'i', 'q7')
    f.addSetTransition('q9', VOWS, 'q5')
    # Return your completed FST
    return f
    



In [ ]:
if __name__ == '__main__':
    testFST(print_examples='incorrect')

3295 / 3295 = 100.0% of examples parsed
+-------------+------------------+----------------------+-----------+
| Input       | Correct Output   | Returned Output      | Result    |
|-------------+------------------+----------------------+-----------|
| menguar     | menguó           | menguóminguó         | INCORRECT |
| atreguar    | atreguó          | atreguóatriguó       | INCORRECT |
| arpegiar    | arpegió          | arpegióarpigió       | INCORRECT |
| privilegiar | privilegió       | privilegiópriviligió | INCORRECT |
+-------------+------------------+----------------------+-----------+

Scorecard:
+-----------------------------------------------+-----------------------------+-----------+---------+----------------+
| Category                                      | Example                     |   Correct |   Total |   Accuracy (%) |
|-----------------------------------------------+-----------------------------+-----------+---------+----------------|
| 0a* Regular -Car verb (stem e